In [4]:
# LangChain 설치
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 11.2 MB/s eta 0:00:00


In [6]:
# 언어 모델 설치
!pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [7]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",  # 사용할 OpenAI 모델의 이름
    temperature=0,          # 언어 모델의 출력값 다양성을 결정하는 값
    max_tokens=None,        # 생성할 토큰의 최대값
    # logprobs=True,        # logprobs 반환 여부
    # stream_options,       # 스트리밍 출력값 설정
    timeout=None,           # 각 요청의 타임아웃
    max_retries=2,          # 재시도 횟수 최대값
    # api_key=""              # OpenAI API 키, 생략시 환경변수 OPENAI_API_KEY 사용
    # base_url=""             # API 요청 URL, 프록시 또는 서비스 에뮬레이터 사용하는 경우만 명시
    # organization=""         # OpenAI organization 아이디, 생략시 환경변수 OPENAI_ORG_ID 사용
    ) # ChatModel

# refer
# https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI
# https://www.iguazio.com/glossary/llm-temperature/

··········


In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian."),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 21, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-acce27b0-4c0d-4c5d-9c45-aa3a0d94edfc-0', usage_metadata={'input_tokens': 21, 'output_tokens': 3, 'total_tokens': 24})

# OutputParsers

In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [10]:
result = model.invoke(messages) # AIMessage

parser.invoke(result)

'Ciao!'

In [11]:
# 위 과정을 체인으로 생성
chain = model | parser
chain.invoke(messages)

'Ciao!'

# Prompt Templates

In [14]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"  # language 변수의 값을 사용

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", "{text}")  # text 변수의 값을 사용
    ]
)

result = prompt_template.invoke(
    {
        "language": "italian",  # language 변수 값 정의
        "text": "hi"            # text 변수 값 정의
    }
)

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

# Chaining together components with LCEL

In [15]:
chain = prompt_template | model | parser
chain.invoke(
    {
        "language": "italian",
        "text": "hi"
    }
)

'Ciao'

# Serving with LangServe

- LangChain의 체인을 REST API로 배포해준다.
- LangChain을 사용하기 위해 LangServe를 사용할 필요는 없지만, LangServe로 당신의 애플리케이션을 배포하는 방법을 다룬다.

In [16]:
!pip install "langserve[all]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.0 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python

from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', system_template),
        ('user', '{text}')
    ]
)

# 2. Create model
model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser

# 5. App definition
app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple API server using LangChain's Runnable interfaces",
)

# 6. Adding chain route
add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
  import uvicorn
  uvicorn.run(app, host="localhost", port=8000)

In [ ]:
python serve.py